# Weather (2)

In [1]:
import pandas as pd

In [2]:
# weather data from https://agriculture.alberta.ca/acis/weather-data-viewer.jsp

In [3]:
df = pd.read_csv('data_and_location2.csv', index_col=0)

In [4]:
df.head()

,FieldID,Date,Year_x,JDay,SsCtMean,SsCtSD,SsMean,logSsMean,TtCt,TtSD,...,Year_y,SamplerNo,CropType,Crop,Region,Province,SamplerPresent,SamplerType,Lat,Long
0,1904,2019-07-05,2019,186,37.15,0.63,6.51,0.875640,23.60,0.10,...,2019.0,NaN,Bean,Bean,Vauxhall,Alberta,Yes,Burkard,49.869673,-112.076528
1,1904,2019-07-06,2019,187,40.00,0.00,0.00,0.000000,23.29,0.07,...,2019.0,NaN,Bean,Bean,Vauxhall,Alberta,Yes,Burkard,49.869673,-112.076528
2,1904,2019-07-07,2019,188,36.85,0.25,7.57,0.932981,23.40,0.07,...,2019.0,NaN,Bean,Bean,Vauxhall,Alberta,Yes,Burkard,49.869673,-112.076528
3,1904,2019-07-08,2019,189,38.80,0.76,2.32,0.521138,24.03,0.05,...,2019.0,NaN,Bean,Bean,Vauxhall,Alberta,Yes,Burkard,49.869673,-112.076528
4,1904,2019-07-09,2019,190,33.44,0.25,65.06,1.819939,24.41,0.08,...,2019.0,NaN,Bean,Bean,Vauxhall,Alberta,Yes,Burkard,49.869673,-112.076528


In [5]:
df['Date'] = pd.to_datetime(df['Date'])

In [6]:
df2 = df.sort_values(by='Date')

In [8]:
df2.tail()

,FieldID,Date,Year_x,JDay,SsCtMean,SsCtSD,SsMean,logSsMean,TtCt,TtSD,...,Year_y,SamplerNo,CropType,Crop,Region,Province,SamplerPresent,SamplerType,Lat,Long
746,2042,2020-08-26,2020,239,NaN,NaN,15165.722660,4.181,NaN,NaN,...,2020.0,NaN,Bean disease nursery,Bean,Lethbridge,Alberta,Yes,Burkard,49.70232,-112.75915
747,2042,2020-08-27,2020,240,NaN,NaN,8115.201511,3.909,NaN,NaN,...,2020.0,NaN,Bean disease nursery,Bean,Lethbridge,Alberta,Yes,Burkard,49.70232,-112.75915
748,2042,2020-08-28,2020,241,NaN,NaN,3191.067340,3.504,NaN,NaN,...,2020.0,NaN,Bean disease nursery,Bean,Lethbridge,Alberta,Yes,Burkard,49.70232,-112.75915
749,2042,2020-08-29,2020,242,NaN,NaN,2569.229289,3.410,NaN,NaN,...,2020.0,NaN,Bean disease nursery,Bean,Lethbridge,Alberta,Yes,Burkard,49.70232,-112.75915
750,2042,2020-08-30,2020,243,NaN,NaN,729.416321,2.864,NaN,NaN,...,2020.0,NaN,Bean disease nursery,Bean,Lethbridge,Alberta,Yes,Burkard,49.70232,-112.75915


In [9]:
# 2018 dates start on June 13, 2020 dates end on Aug 30
df2['FieldID'].unique()

array(['Vauxhall', 'Cranford', 'Taber', '1919', '1920', '1921', '1906',
       '1910', '1922', '1904', '1917', '1915', '2007', '2006', '2042',
       '2001', '2004'], dtype=object)

In [10]:
# vauxhall = FieldID 1814
# Taber = FieldID 1804
# Cranford = FieldID 1810

rolling hills weather: 'vauxhall' \
barnwell weather: 'cranford' \
lethbridge weather: 1919, 1920, 1921, 1922, 2042 \
vauxhall weather: 1906, 1910, 2007 \
fincastle weather: 1904 \
bow weather = 1917, 2006, 2004, 'taber' \
bow north weather = 1915 \
grassy lake weather: 2001

In [211]:
# create dew point function
def dew_point(RH, Temp):
    a = Temp
    b = ((100-RH)/5)
    DP = a-b
    return DP

def rolling_mean(data, col):
    # -1 day data
    name1 = col + '_1'
    # shift by 1
    lag1 = data[col].shift(1)
    # assign to column  
    data[name1] = lag1
    
    # -3 day mean
    name3 = col + '_3'
    # assign mean to column
    window3 = lag1.rolling(window=3).mean()
    data[name3] = window3
    
    # 7 day rolling mean leads to a lot of lost values
    # -7 day mean
    name7 = col + '_7'
    # assign mean to column
    window7 = lag1.rolling(window=7).mean()
    data[name7] = window7
    
    # fill NaN's with june mean info so that we can keep as much data as possible
   # data[name1] = data[name1].fillna(value=data.iloc[1:7][name1].mean())
    #data[name3] = data[name3].fillna(value=data.iloc[6:10][name3].mean())
    #data[name7] = data[name7].fillna(value=data.iloc[13:20][name7].mean())
    
    # return new dataframe
    return data

def rolling_sum(data, col):
    # -1 day data
    name1 = col + 'Sum_1'
    # shift by 1
    lag1 = data[col].shift(1)
    # assign to column  
    data[name1] = lag1
    
    # -3 day mean
    name3 = col + 'Sum_3'
    # assign mean to column
    window3 = lag1.rolling(window=3).sum()
    data[name3] = window3
    
    # 7 day rolling mean leads to a lot of lost values
    # -7 day mean
    name7 = col + 'Sum_7'
    # assign mean to column
    window7 = lag1.rolling(window=7).sum()
    data[name7] = window7
    
    # fill NaN's with june mean info so that we can keep as much data as possible
   # data[name1] = data[name1].fillna(value=data.iloc[1:7][name1].mean())
    #data[name3] = data[name3].fillna(value=data.iloc[6:10][name3].mean())
    #data[name7] = data[name7].fillna(value=data.iloc[13:20][name7].mean())
    
    # return new dataframe
    return data

def temp_DP(data, col):
    name = col + '_uDP'
    data[name] = data.apply(lambda x: 1 if x[col] <= x['MeanDP'] else 0, axis = 1)
    return data

def cool_temp(data, col):
    name = col + '_u20'
    data[name] = data[col].apply(lambda x: 1 if x <= 20 else 0)
    return data

In [212]:
# wrap all feature engineering into 1 function
def engineer(data):
    # drop extra columns
    to_drop = ['Air Temp. Min. Source Flag', 'Air Temp. Min. Record Completeness (%)',
       'Air Temp. Max. Source Flag','Air Temp. Max. Record Completeness (%)', 
       'Air Temp. Avg. Source Flag', 'Air Temp. Avg. Record Completeness (%)',
       'Humidity Avg. Source Flag','Humidity Avg. Record Completeness (%)', 
       'Precip. Accumulated Source Flag', 'Precip. Accumulated Comment','Precip. Source Flag', 'Precip. Comment',
        'Wind Speed 2 m Avg. Source Flag','Wind Speed 2 m Avg. Record Completeness (%)',
        'Wind Speed 10 m Avg. Source Flag','Wind Speed 10 m Avg. Record Completeness (%)',
        'Wind Dir. 10 m Avg. Source Flag','Wind Dir. 10 m Avg. Record Completeness (%)',
       'Frost Probability 0�C Source Flag','Frost Probability 0�C Record Completeness (%)', 
        'Precip. Accumulated (mm)']
    data.drop(labels=to_drop, axis=1, inplace=True)
    # rename columns
    data.columns = ['Station','Date','MinTemp','MaxTemp','MeanTemp','MeanRH','Precip','Wind2','Wind10','Wind10Dir','Frost']
    # turn date column into datetime
    data['Date'] = pd.to_datetime(data['Date'])
    # create dew point feature
    data['MeanDP'] = dew_point(data['MeanRH'], data['MeanTemp'])
    # create counts below dewpoint and 20 degrees
    temp_cols = ['MinTemp', 'MaxTemp', 'MeanTemp']
    for col in temp_cols:
        data = temp_DP(data, col)
        data = cool_temp(data, col)
    # calculate rolling means and sums
    roll_means = ['MinTemp','MaxTemp','MeanTemp','MeanRH','Precip','Wind2','Wind10','Wind10Dir','Frost','MeanDP']
    roll_sums = ['Precip','MinTemp_uDP','MaxTemp_uDP','MeanTemp_uDP', 'MinTemp_u20', 'MaxTemp_u20', 'MeanTemp_u20']
    for col in roll_means:
        data = rolling_mean(data, col)
    for col in roll_sums:
        data = rolling_sum(data, col)
    return data

In [213]:
weather_dfs = []

In [214]:
rolling_hills1 = pd.read_csv('rolling_hills_18-19.csv')
rolling_hills2 = pd.read_csv('rolling_hills_20.csv')
rolling_hills = pd.concat([rolling_hills1, rolling_hills2], ignore_index=True)
weather_dfs.append(rolling_hills)

In [215]:
barnwell1 = pd.read_csv('barnwell_18-19.csv')
barnwell2 = pd.read_csv('barnwell_20.csv')
barnwell = pd.concat([barnwell1, barnwell2], ignore_index=True)
weather_dfs.append(barnwell)

In [216]:
lethbridge1 = pd.read_csv('lethbridge_18-19.csv')
lethbridge2 = pd.read_csv('lethbridge_20.csv')
lethbridge = pd.concat([lethbridge1, lethbridge2], ignore_index=True)
weather_dfs.append(lethbridge)

In [217]:
vauxhall1 = pd.read_csv('vauxhall_18-19.csv')
vauxhall2 = pd.read_csv('vauxhall_20.csv')
vauxhall = pd.concat([vauxhall1, vauxhall2], ignore_index=True)
weather_dfs.append(vauxhall)

In [218]:
fincastle1 = pd.read_csv('fincastle_18-19.csv')
fincastle2 = pd.read_csv('fincastle_20.csv')
fincastle = pd.concat([fincastle1, fincastle2], ignore_index=True)
weather_dfs.append(fincastle)

In [219]:
bow1 = pd.read_csv('bow_18-19.csv')
bow2 = pd.read_csv('bow_20.csv')
bow = pd.concat([bow1, bow2], ignore_index=True)
weather_dfs.append(bow)

In [220]:
bow_north1 = pd.read_csv('bow_north_18-19.csv')
bow_north2 = pd.read_csv('bow_north_20.csv')
bow_north = pd.concat([bow_north1, bow_north2], ignore_index=True)
weather_dfs.append(bow_north)

In [221]:
grassy_lake1 = pd.read_csv('grassy_lake_18-19.csv')
grassy_lake2 = pd.read_csv('grassy_lake_20.csv')
grassy_lake = pd.concat([grassy_lake1, grassy_lake2], ignore_index=True)
weather_dfs.append(grassy_lake)

In [222]:
location_names = ['rolling_hills','barnwell','lethbridge','vauxhall','fincastle','bow','bow_north','grassy_lake']
for frame in weather_dfs:
    index = 0
    frame = engineer(frame)
    filename = location_names[index]
    frame.to_csv(filename + '2_processed.csv')
    index += 1

In [223]:
rolling_hills.shape

(600, 69)

In [197]:
rolling_hills.head()

,Station,Date,MinTemp,MaxTemp,MeanTemp,MeanRH,Precip,Wind2,Wind10,Wind10Dir,...,MeanTemp_uDPSum_7,MinTemp_u20Sum_1,MinTemp_u20Sum_3,MinTemp_u20Sum_7,MaxTemp_u20Sum_1,MaxTemp_u20Sum_3,MaxTemp_u20Sum_7,MeanTemp_u20Sum_1,MeanTemp_u20Sum_3,MeanTemp_u20Sum_7
0,Rolling Hills AGCM,2018-05-27,5.6,27.9,18.7,57.0,0.0,9.9,11.5,68.1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Rolling Hills AGCM,2018-05-28,9.5,31.6,21.8,44.8,0.0,16.3,20.1,182.7,...,NaN,1.0,NaN,NaN,0.0,NaN,NaN,1.0,NaN,NaN
2,Rolling Hills AGCM,2018-05-29,11.1,22.1,17.3,42.3,0.0,19.9,25.3,341.5,...,NaN,1.0,NaN,NaN,0.0,NaN,NaN,0.0,NaN,NaN
3,Rolling Hills AGCM,2018-05-30,7.9,20.4,13.0,65.4,6.1,13.0,15.9,344.6,...,NaN,1.0,3.0,NaN,0.0,0.0,NaN,1.0,2.0,NaN
4,Rolling Hills AGCM,2018-05-31,4.2,10.2,8.5,92.5,5.2,14.8,18.2,40.0,...,NaN,1.0,3.0,NaN,0.0,0.0,NaN,1.0,2.0,NaN


### Merge weather location dfs with main dfs

In [259]:
df_rolling_hills = df2[df2['FieldID'] == 'Vauxhall'].copy()
df_barnwell = df2[df2['FieldID'] == 'Cranford'].copy()
df_lethbridge = df2[(df2['FieldID'] == '1919') | (df2['FieldID'] == '1920') | (df2['FieldID'] == '1921') | (df2['FieldID'] == '1922') | (df2['FieldID'] == '2042')].copy()
df_vauxhall = df2[(df2['FieldID'] == '1906') | (df2['FieldID'] == '1910') | (df2['FieldID'] == '2007')].copy()
df_fincastle = df2[df2['FieldID'] == '1904'].copy()
df_bow = df2[(df2['FieldID'] == '1917') | (df2['FieldID'] == '2006') | (df2['FieldID'] == '2004') | (df2['FieldID'] == 'Taber')].copy()
df_bow_north = df2[df2['FieldID'] == '1915'].copy()
df_grassy_lake = df2[df2['FieldID'] == '2001'].copy()

In [260]:
df_rolling_hills.shape

(77, 85)

In [205]:
df_rolling_hills.head()

,FieldID,Date,Year_x,JDay,SsCtMean,SsCtSD,SsMean,logSsMean,TtCt,TtSD,...,Year_y,SamplerNo,CropType,Crop,Region,Province,SamplerPresent,SamplerType,Lat,Long
905,Vauxhall,2018-06-13,2018,164,NaN,NaN,NaN,NaN,NaN,NaN,...,2019.0,NaN,City,NaN,Vauxhall,Alberta,No,NaN,50.070242,-112.106166
906,Vauxhall,2018-06-14,2018,165,NaN,NaN,NaN,NaN,NaN,NaN,...,2019.0,NaN,City,NaN,Vauxhall,Alberta,No,NaN,50.070242,-112.106166
907,Vauxhall,2018-06-15,2018,166,NaN,NaN,NaN,NaN,NaN,NaN,...,2019.0,NaN,City,NaN,Vauxhall,Alberta,No,NaN,50.070242,-112.106166
908,Vauxhall,2018-06-16,2018,167,NaN,NaN,NaN,NaN,NaN,NaN,...,2019.0,NaN,City,NaN,Vauxhall,Alberta,No,NaN,50.070242,-112.106166
909,Vauxhall,2018-06-17,2018,168,NaN,NaN,14.95,1.202761,NaN,NaN,...,2019.0,NaN,City,NaN,Vauxhall,Alberta,No,NaN,50.070242,-112.106166


In [256]:
#['rolling_hills','barnwell','lethbridge','vauxhall','fincastle','bow','bow_north','grassy_lake']
data_dfs = [df_rolling_hills, df_barnwell, df_lethbridge,df_vauxhall, df_fincastle, df_bow, df_bow_north, df_grassy_lake]

for i in range(len(data_dfs)):
    frame = data_dfs[i]
    weather = weather_dfs[i]
    frame = frame.merge(weather, how='left', on='Date', copy=False)
    data_dfs[i] = frame

In [243]:
data_dfs[0] = data_dfs[0].merge(weather_dfs[0], how='left', on='Date')

In [257]:
data_dfs[0].shape

(77, 153)

In [258]:
df_rolling_hills.shape

(77, 85)

In [203]:
df_rolling_hills.head()

,FieldID,Date,Year_x,JDay,SsCtMean,SsCtSD,SsMean,logSsMean,TtCt,TtSD,...,MeanTemp_uDPSum_7,MinTemp_u20Sum_1,MinTemp_u20Sum_3,MinTemp_u20Sum_7,MaxTemp_u20Sum_1,MaxTemp_u20Sum_3,MaxTemp_u20Sum_7,MeanTemp_u20Sum_1,MeanTemp_u20Sum_3,MeanTemp_u20Sum_7
0,Vauxhall,2018-06-13,2018,164,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,1.0,3.0,7.0,0.0,2.0,2.0,1.0,3.0,6.0
1,Vauxhall,2018-06-14,2018,165,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,1.0,3.0,7.0,0.0,1.0,2.0,1.0,3.0,6.0
2,Vauxhall,2018-06-15,2018,166,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,1.0,3.0,7.0,0.0,0.0,2.0,1.0,3.0,6.0
3,Vauxhall,2018-06-16,2018,167,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,1.0,3.0,7.0,0.0,0.0,2.0,1.0,3.0,7.0
4,Vauxhall,2018-06-17,2018,168,NaN,NaN,14.95,1.202761,NaN,NaN,...,0.0,1.0,3.0,7.0,1.0,1.0,3.0,1.0,3.0,7.0


In [115]:
main = pd.concat([cran_df2, taber_df2])

In [116]:
main.shape

(208, 107)

In [117]:
gl_df = df.loc[(df['Location.x'] == '1915') | (df['Location.x'] == '1917')]

In [118]:
gl_df.shape

(100, 98)

In [119]:
gl['Date (Local Standard Time)'] = pd.to_datetime(gl['Date (Local Standard Time)'])

gl_df2 = gl_df.merge(gl, how='left', left_on='Date', right_on='Date (Local Standard Time)')

In [120]:
gl_df2.shape

(100, 107)

In [121]:
main = pd.concat([main, gl_df2])

In [122]:
main.shape

(308, 107)

In [123]:
v_df = df.loc[(df['Location.x'] == '1906') | (df['Location.x'] == '1910') | (df['Location.x'] == 'Vauxhall')]

In [124]:
v_df.shape

(196, 98)

In [125]:
vaux['Date (Local Standard Time)'] = pd.to_datetime(vaux['Date (Local Standard Time)'])

v_df2 = v_df.merge(vaux, how='left', left_on='Date', right_on='Date (Local Standard Time)')

In [126]:
v_df2.shape

(196, 107)

In [127]:
main = pd.concat([main, v_df2])

In [128]:
main.shape

(504, 107)

In [129]:
n_df = df.loc[df['Location.x'] == '1919']

In [130]:
n_df.shape

(64, 98)

In [131]:
nine['Date (Local Standard Time)'] = pd.to_datetime(nine['Date (Local Standard Time)'])

n_df2 = n_df.merge(nine, how='left', left_on='Date', right_on='Date (Local Standard Time)')

In [132]:
n_df2.shape

(64, 107)

In [133]:
main = pd.concat([main, n_df2])

In [134]:
main.shape

(568, 107)

In [135]:
df.shape

(568, 98)

In [136]:
main.to_csv('final_df.csv')

In [137]:
main.head()

,Location.x,JDay,Date,SsCtMean,SsCtSD,SsMean,logSsMean,SsSD,SsClean,logSsClean,...,Long,Station Name,Date (Local Standard Time),Air Temp. Min. (�C),Air Temp. Max. (�C),Air Temp. Avg. (�C),Humidity Avg. (%),Precip. (mm),Wind Speed 10 m Avg. (km/h),Frost Probability 0�C (%)
0,Cranford,164,2018-06-13,NaN,NaN,295.035,2.471343,NaN,NaN,NaN,...,NaN,Barnwell AGDM,2018-06-13,7.7,26.5,17.9,38.4,0.0,22.6,0.0
1,Cranford,165,2018-06-14,NaN,NaN,106.900,2.033021,NaN,NaN,NaN,...,NaN,Barnwell AGDM,2018-06-14,4.6,22.0,13.1,65.9,1.2,19.1,0.0
2,Cranford,166,2018-06-15,NaN,NaN,164.775,2.219519,NaN,NaN,NaN,...,NaN,Barnwell AGDM,2018-06-15,7.2,22.0,13.2,77.8,7.2,11.1,0.0
3,Cranford,167,2018-06-16,NaN,NaN,138.380,2.144200,NaN,NaN,NaN,...,NaN,Barnwell AGDM,2018-06-16,11.2,18.1,12.5,89.3,9.8,16.5,0.0
4,Cranford,168,2018-06-17,NaN,NaN,154.545,2.191856,NaN,NaN,NaN,...,NaN,Barnwell AGDM,2018-06-17,7.7,21.8,15.8,67.1,0.0,7.8,0.0


In [139]:
main.isnull().sum().sort_values(ascending=False).head(10)

SamplerNo        568
DiffMinSoilT     333
DiffMeanSoilT    333
DiffMaxSoilT     333
MaxSoilT_1d      331
MinSoilT_1d      331
MeanSoilT_1d     331
MaxSoilTemp      327
MeanSoilTemp     327
MinSoilTemp      327
dtype: int64

In [140]:
main.columns

Index(['Location.x', 'JDay', 'Date', 'SsCtMean', 'SsCtSD', 'SsMean',
       'logSsMean', 'SsSD', 'SsClean', 'logSsClean',
       ...
       'Long', 'Station Name', 'Date (Local Standard Time)',
       'Air Temp. Min. (�C)', 'Air Temp. Max. (�C)', 'Air Temp. Avg. (�C)',
       'Humidity Avg. (%)', 'Precip. (mm)', 'Wind Speed 10 m Avg. (km/h)',
       'Frost Probability 0�C (%)'],
      dtype='object', length=107)

In [141]:
df.columns

Index(['Location.x', 'JDay', 'Date', 'SsCtMean', 'SsCtSD', 'SsMean',
       'logSsMean', 'SsSD', 'SsClean', 'logSsClean', 'TtCt', 'TtSD', 'Water.',
       'logSsMean_t1', 'Year_x', 'MeanWetness', 'DiffMeanWet', 'MeanWet_1d',
       'MaxWetness', 'DiffMaxWet', 'MaxWet_1d', 'MinWetness', 'DiffMinWet',
       'MinWet_1d', 'MeanTemp', 'DiffMeanT', 'MeanT_1d', 'MaxTemp', 'DiffMaxT',
       'MaxT_1d', 'MinTemp', 'DiffMinT', 'MinT_1d', 'MeanRH', 'DiffMeanRH',
       'MeanRH_1d', 'MaxRH', 'DiffMaxRH', 'MaxRH_1d', 'MinRH', 'DiffMinRH',
       'MinRH_1d', 'DiffRH_0d', 'DiffRH_1d', 'DiffRH_2d', 'MaxDiffRH_2h',
       'MaxDiffRH_3h', 'MeanVPD', 'DiffMeanVPD', 'MeanVPD_1d', 'MaxVPD',
       'DiffMaxVPD', 'MaxVPD_1d', 'MinVPD', 'DiffMinVPD', 'TotalPrecip',
       'Precip_1d', 'RainYN', 'MaxRain', 'MinRain', 'MeanWC', 'DiffMeanWC',
       'MeanWC_1d', 'MaxWC', 'DiffMaxWC', 'MaxWC_1d', 'MinWC', 'DiffMinWC',
       'MinWC_1d', 'MeanDP', 'DiffMeanDP', 'MeanDP_1d', 'MaxDP', 'DiffMaxDP',
       'MaxDP_1d'

In [142]:
to_drop = ['SamplerNo', 'Date (Local Standard Time)', 'Station Name', 'SamplerPresent','SamplerType','Province','Region','Crop','CropType','FieldID']

main.drop(labels=to_drop, axis=1, inplace=True)

In [143]:
main.shape

(568, 97)

In [144]:
main.to_csv('final_df.csv')